In [1]:
import tensorflow as tf
print("TensorFlow version is ", tf.__version__)
import numpy as np
keras=tf.keras
from src import input_fn
import matplotlib
import matplotlib.pyplot as plt
import sys
from tensorflow.keras.models import load_model
from src import config as cfg

import pandas as pd
import os
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils
import imutils
import cv2
from src.gradcam import GradCAM

TensorFlow version is  2.0.0


In [2]:
#input files
train = cfg.input_file['train']
test = cfg.input_file['test']

# Model Parameters
batch_size = cfg.chexper_params['batch_size']
lr =  cfg.chexper_params['lr']
epoches = cfg.chexper_params['epoches']
image_size = cfg.chexper_params['image_size']

In [3]:
#labeling approach 
approach= 'u-Ones'

In [15]:
col_names=['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'PleuralEffusion']

In [5]:
test_x,test_y=input_fn.process_testing_data(file_path=test)
test_input= input_fn.input_fn_multi_output(False,test_x, None,batch_size)

In [9]:
import efficientnet.tfkeras


model = load_model('uOnes.h5', compile=False)

In [10]:
#originals = cv2.imread('../CheXpert-v1.0-small/valid/patient64541/study1/view1_frontal.jpg')

originals = []
for path in test_x:
    originals.append(cv2.resize(cv2.imread(path), (320,320)))
print(len(originals))

234


In [11]:
print(test_input)


<PrefetchDataset shapes: (None, 320, 320, 3), types: tf.float32>


In [12]:
# image = load_img('../CheXpert-v1.0-small/valid/patient64555/study1/view1_frontal.jpg', target_size=(320,320))
# image = img_to_array(image)
# image = np.expand_dims(image, axis=0)
# image = imagenet_utils.preprocess_input(image)
# test_lst = np.array([k for k in test_input])
# image = test_lst[0][1]
# 
# print(image.shape)
batches = np.array([x for x in test_input])
images = np.array(batches[0])
for batch in range(1, len(batches)):
    images = np.concatenate((images, np.array(batches[batch])), 0)
    
print(images.shape)
print(images[0].shape)

(234, 320, 320, 3)
(320, 320, 3)


In [16]:
for indx in range(len(images)):
    print("Generating CAM for image {}/{}".format(indx+1, len(images)))
    image = images[indx]
    orig = originals[indx]
    image = np.expand_dims(image, axis=0)
    preds = model.predict(image)
    i = np.argmax(preds)
    label = col_names[i]
    cam = GradCAM(model, i)
    heatmap = cam.compute_heatmap(image)
    heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
    (heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)
    cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
    cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,0.8, (255, 255, 255), 2)
    output = np.vstack([orig, heatmap, output])
    output = imutils.resize(output, height=700)
    filename = "cams_output\\image_{}_{}.png".format(indx+1, approach)
    cv2.imwrite(filename, output)

Generating CAM for image 1/234
Generating CAM for image 2/234
Generating CAM for image 3/234
Generating CAM for image 4/234
Generating CAM for image 5/234
Generating CAM for image 6/234
Generating CAM for image 7/234
Generating CAM for image 8/234
Generating CAM for image 9/234
Generating CAM for image 10/234
Generating CAM for image 11/234
Generating CAM for image 12/234
Generating CAM for image 13/234
Generating CAM for image 14/234
Generating CAM for image 15/234
Generating CAM for image 16/234
Generating CAM for image 17/234
Generating CAM for image 18/234
Generating CAM for image 19/234
Generating CAM for image 20/234
Generating CAM for image 21/234
Generating CAM for image 22/234
Generating CAM for image 23/234
Generating CAM for image 24/234
Generating CAM for image 25/234
Generating CAM for image 26/234
Generating CAM for image 27/234
Generating CAM for image 28/234
Generating CAM for image 29/234
Generating CAM for image 30/234
Generating CAM for image 31/234
Generating CAM fo

In [1]:
!jupyter nbconvert --to html 03_Visualization_ClassActivationMaps.ipynb

[NbConvertApp] Converting notebook 03_Visualization_ClassActivationMaps.ipynb to html
[NbConvertApp] Writing 298422 bytes to 03_Visualization_ClassActivationMaps.html
